In [97]:
import lhapdf
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.interpolate import interp1d

In [426]:
CF = 4/3
CA = 3
Nc = 3
TR = .5

p = lhapdf.mkPDF("cteq6l1") 
def alphas(Q2):
    return p.alphasQ2(Q2)


LHAPDF 6.4.0 loading /home/weiyaoke/.local/share/LHAPDF/cteq6l1/cteq6l1_0000.dat
cteq6l1 PDF set, member #0, version 4; LHAPDF ID = 10042


In [427]:
# LO XXX * 1/v *delta(1-w)
# the XXX:

# q+Q->q+Q
def qQ2qQ_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2*CF/Nc/s*(1+v**2)/(1-v)**2

# Q+Qbar->q+qbar
def QQbar2qqbar_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2*CF/Nc/s * (2*v**2-2*v+1)

# qq->qq
def qq2qq_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2*2*CF/Nc**2/s\
    * (Nc*v**4-2*Nc*v**3+(4*Nc+1)*v**2-(3*Nc+1)*v+Nc)/(1-v)**2/v**2

# qqbar->qqbar
def qqbar2qqbar_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2*2*CF/Nc**2/s\
    * (Nc*v**4-(3*Nc+1)*v**3+(4*Nc+1)*v**2-2*Nc*v**2+Nc)/(1-v)**2

# qg->qg
def qg2qg_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2/2/Nc**2/s/v/(1-v)**2\
        * (1+v**2)*( (Nc**2-1)*(1+v**2) + 2*v )

# gg->qqbar
def gg2qqbar_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2/2/Nc/(Nc**2-1)/s/v/(1-v)*(v**2+(1-v)**2)\
        * ( Nc**2-1 - 2*Nc**2*v*(1-v) )

# qqbar -> gg
def qqbar2gg_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2*CF/Nc**2/s*(1-2*v+2*v**2)*(Nc**2-1 - 2*Nc**2*v*(1-v))

# gg -> gg
def gg2gg_dv(s, v, muR2):
    a2 = alphas(muR2)**2
    return np.pi*a2*4*Nc**2/(Nc**2-1)/s*(1-v+v**2)**3/(v**2*(1-v)**2)

In [428]:
# LO AB->q
def dsigma_AB2q_pTdpT_dy_dphi(pT, y, pid, mur=1):
    alldiff_qids = [-5,-4,-3,-2,-1,1,2,3,4,5]
    alldiff_qids.remove(pid)
    
    alldiff_flavor = [1,2,3,4,5]
    alldiff_flavor.remove(np.abs(pid))

    # beam1 = ss/2, 0, 0, ss/2
    # beam2 = ss/2, 0, 0,-ss/2
    # the outgoing parton (set D(z)=delta(1-z))
    # pJ = pT*chy, pT, 0, pT*shy
    # S = (beam1+beam2)^2, 
    # T = (beam1-pJ)^2 
    # T = (beam2-pJ)^2 

    S = sqrts**2
    T = (sqrts/2-pT*np.cosh(y))**2 - pT**2 - (sqrts/2-pT*np.sinh(y))**2
    U = (sqrts/2-pT*np.cosh(y))**2 - pT**2 - (-sqrts/2-pT*np.sinh(y))**2
    V = 1+T/S
    W = -U/(S+T)
    
    if pT*np.cosh(y)>sqrts/2:
        return 0
    
    # LO: delta(1-w) dw --> w=1
    def df(k):
        v = 1/(1+np.exp(-k))
        
        x1 = V*W/v
        x2 = (1-V)/(1-v)
        if x1>1 or x2>1 or x1<0 or x2<0:
            return 0
        s = S*x1*x2
        muR2 = s*mur**2
        muF2 = s*mur**2
        
        res = 0
        
        # q + Q --> q + X
        F1F2 = p.xfxQ2(pid, x1, muF2)/x1 \
             * np.sum([p.xfxQ2(sid, x2, muF2)/x2 for sid in alldiff_qids])
        ds = qQ2qQ_dv(s, v, muR2)
        res += ds*F1F2
        
        # Q + q --> q + X
        F1F2 = np.sum([p.xfxQ2(sid, x1, muF2)/x1 for sid in alldiff_qids])\
             * p.xfxQ2(pid, x2, muF2)/x2
        ds = qQ2qQ_dv(s, 1-v, muR2)
        res += ds*F1F2
        
        # Q + Qbar --> q + X
        F1F2 = np.sum([p.xfxQ2(p1, x1, muF2)/x1*p.xfxQ2(-p1, x2, muF2)/x2\
                      +p.xfxQ2(-p1, x1, muF2)/x1*p.xfxQ2(p1, x2, muF2)/x2\
                       for p1 in alldiff_flavor])
        ds = QQbar2qqbar_dv(s, v, muR2)
        res += ds*F1F2
        
        # q + q --> q + X
        F1F2 = p.xfxQ2(pid, x1, muF2)/x1*p.xfxQ2(pid, x2, muF2)/x2
        ds = qq2qq_dv(s, v, muR2)
        res += ds*F1F2
        
        # q + qbar --> q + X
        F1F2 = p.xfxQ2(pid, x1, muF2)/x1*p.xfxQ2(-pid, x2, muF2)/x2
        ds = qqbar2qqbar_dv(s, v, muR2)
        res += ds*F1F2
        
        # q + qbar --> qbar + X
        F1F2 = p.xfxQ2(-pid, x1, muF2)/x1*p.xfxQ2(pid, x2, muF2)/x2
        ds = qqbar2qqbar_dv(s, 1-v, muR2)
        res += ds*F1F2
        
        # q + g --> q + X
        F1F2 = p.xfxQ2(pid, x1, muF2)/x1*p.xfxQ2(21, x2, muF2)/x2
        ds = qg2qg_dv(s, v, muR2)
        res += ds*F1F2
        
        # g + q --> q + X
        F1F2 = p.xfxQ2(21, x1, muF2)/x1*p.xfxQ2(pid, x2, muF2)/x2
        ds = qg2qg_dv(s, 1-v, muR2)
        res += ds*F1F2
        
        # g + g --> q + X
        F1F2 = p.xfxQ2(21, x1, muF2)/x1*p.xfxQ2(21, x2, muF2)/x2
        ds = gg2qqbar_dv(s, v, muR2)
        res += ds*F1F2
        
        return res
    
    vmax = V
    vmin = V*W
    
    GeVm2_to_mb = 1/5.076**2/100*1000
    return GeVm2_to_mb/np.pi/S * quad(df, np.log(vmin/(1-vmin)), np.log(vmax/(1-vmax)), epsrel=.01)[0] 

In [429]:
# LO: AB->g
def dsigma_AB2g_pTdpT_dy_dphi(pT, y, mur=1):    
    qs = [-5,-4,-3,-2,-1,1,2,3,4,5]
    fs = [1,2,3,4,5]

    if pT*np.cosh(y)>sqrts/2:
        return 0

    S = sqrts**2
    T = (sqrts/2-pT*np.cosh(y))**2 - pT**2 - (sqrts/2-pT*np.sinh(y))**2
    U = (sqrts/2-pT*np.cosh(y))**2 - pT**2 - (-sqrts/2-pT*np.sinh(y))**2
    V = 1+T/S
    W = -U/(S+T)
    
    # LO: delta(1-w) dw --> w=1
    def df(k):
        v = 1/(1+np.exp(-k))
        
        x1 = V*W/v
        x2 = (1-V)/(1-v)
        
        if x1>1 or x2>1 or x1<0 or x2<0:
            return 0

        s = S*x1*x2
        muR2 = s*mur**2
        muF2 = s*mur**2
        
        res = 0
        
        # g+g --> g + X
        F1F2 = p.xfxQ2(21, x1, muF2)/x1 * p.xfxQ2(21, x2, muF2)/x2
        ds = gg2gg_dv(s, v, muR2)
        res += ds*F1F2
        
        # q+qbar --> g + X
        F1F2 = np.sum([p.xfxQ2(pid, x1, muF2)/x1 * p.xfxQ2(-pid, x2, muF2)/x2
                     + p.xfxQ2(-pid, x1, muF2)/x1 * p.xfxQ2(pid, x2, muF2)/x2 for pid in fs])
        ds = qqbar2gg_dv(s, v, muR2)
        res += ds*F1F2
        
        # q+g --> g + X
        F1F2 = np.sum([p.xfxQ2(f, x1, muF2)/x1 for f in qs]) * p.xfxQ2(21, x2, muF2)/x2
        ds = qg2qg_dv(s, 1-v, muR2)
        res += ds*F1F2
        
        # g+q --> g + X
        F1F2 = p.xfxQ2(21, x1, muF2)/x1 * np.sum([p.xfxQ2(f, x2, muF2)/x2 for f in qs])
        ds = qg2qg_dv(s, v, muR2)
        res += ds*F1F2
        
        
        return res
    
    vmax = V
    vmin = V*W
    
    GeVm2_to_mb = 1/5.076**2/100*1000
    return GeVm2_to_mb/np.pi/S * quad(df, np.log(vmin/(1-vmin)), np.log(vmax/(1-vmax)), epsrel=.01)[0] 

In [450]:
sqrts = 200
mu=1
DY = 0.6
Ny=5
dy = DY/Ny



Y = np.linspace(0.2+dy/2, 0.8-dy/2, Ny)
pTs = np.exp(np.linspace(np.log(5),np.log(sqrts/2),30))

Q = []
G = []
for pT in pTs:
    Q.append ( 
        np.sum([np.sum([dsigma_AB2q_pTdpT_dy_dphi(pT, y, pid)  
                for pid in [1,2,3,4,5,-1,-2,-3,-4,-5]])*  pT * 2*np.pi
        for y in Y])*dy/DY
    )

    G.append ( 
        np.sum([dsigma_AB2g_pTdpT_dy_dphi(pT, y)*  pT * 2*np.pi for y in Y])*dy/DY
    )

In [451]:

with open("LO/{ss}/pp_mu{scale}_ALICE_y0d3.dat".format(ss=sqrts, scale=mu),'w') as f:
    for ipT,g,q in zip(pTs, G, Q):
        f.write(("{:1.5e}\t"*3+'\n').format(ipT,g,q))


In [432]:
pT,g,d,dbar,u,ubar,s,sbar,c,b,gamma = np.loadtxt("../../Documents/New-DGLAP/Raa/NLO/pp/pp.all.5.02TeV.y0.SLRMn"
                                                 , skiprows=1).T

In [433]:
lnFd = interp1d(np.log(pT), np.log(g), fill_value='extrapolate', bounds_error=False)

In [434]:
qT = 400
np.exp(lnFd(np.log(qT))) * qT* 2*np.pi

3.361649909240202e-10

In [435]:
dsigma_AB2g_pTdpT_dy_dphi(qT, 0)*  qT * 2*np.pi

1.5886624963729787e-11

In [452]:
GeVm2_to_mb

0.38811173395282184